# Emerald Model Demo
The purpose of this notebook is to demonstrate how the Emerald algorithms process data internally.
The Emerald model has two primary functions:

- __Process:__ Given a database table, this function processes each column by finding its data classification (`named_entity`, `numeric_or_id` or `text`) and computing a PII-free representation which can be used for similarity comparisons.
- __Search:__ Given a valid query of a column name, this function finds all the columns which contain similar data to the query column.

We provide 4 sample datasets containing similar columns for evaluating the effectiveness of our model.

## Imports & Installs

In [1]:
#!python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import emerald

## Load sample data

In [3]:
sample_tables = {}
sample_tables["table_1"] = pd.read_csv("data/sample_1.csv")
sample_tables["table_2"] = pd.read_csv("data/sample_2.csv")
sample_tables["table_3"] = pd.read_csv("data/sample_3.csv")
sample_tables["table_4"] = pd.read_csv("data/sample_4.csv")

In [4]:
print("SAMPLE TABLE 1")
print("--------------")
print("Rows:    {:4d}".format(sample_tables["table_1"].shape[0]))
print("Columns: {:4d}".format(sample_tables["table_1"].shape[1]))
sample_tables["table_1"].head()

SAMPLE TABLE 1
--------------
Rows:     500
Columns:   10


,CCID,ACCOUNT TYPE,SSN,GIVEN NAME,LAST NAME,PHYSICAL ADDRESS,CITY,STATE,POSTAL CODE,GENDER
0,9000007856409346,HIGH INTEREST CHECKING,805004325,JANIS,WELLS,142 WINTERSTEIN DR,FOLSOM,CA,95630,Female
1,9000007787145712,CitiGold Interest Checking,806008628,CHERYL,ISKIN,1763 RED ROCK CT,WESTLAKE VILLAGE,CA,91362,Male
2,9000007789022796,4 YEAR CD,920812461,ARTHUR,MATYURIN,25935 ROLLING HILLS RD,TORRANCE,CA,90505,Female
3,9000007854230503,EQUITY SOURCE-RESIDENTIAL,909159250,SIGNE,BAZILE,892 SPRING AVE,UNIONDALE,NY,11553,Male
4,9000007838746616,VISA CARD,989851692,ALMA,THURTELL,29 VILLAGE CV,WARWICK,NY,10990,Male


In [5]:
print("SAMPLE TABLE 2")
print("--------------")
print("Rows:    {:4d}".format(sample_tables["table_2"].shape[0]))
print("Columns: {:4d}".format(sample_tables["table_2"].shape[1]))
sample_tables["table_2"].head()

SAMPLE TABLE 2
--------------
Rows:     424
Columns:   11


,CCID,UPOF_LONG_NME,P_GIVEN_NAME_ONE,P_LAST_NAME,SOCIAL_SECURITY,P_PHYSICAL_ADDR_LINE_ONE,P_PHYSICAL_CITY_NAME,P_PHYSICAL_PROVINCE_STATE_NAME,P_PHYSICAL_POSTAL_CODE,P_RACE_NAME,P_GEND
0,9.000010e+15,REGULAR CHECKING,JEAN,HAMILTON,178-28-3635,813 W WHITTIER BLVD,MONTEBELLO,CA,90640,American Indian,Other
1,9.000010e+15,Citibank(R) Savings Plus Account,CHARLES,NEGRETE,435-41-1162,4166 ALMOND ST,RIVERSIDE,CA,92501,White,Female
2,9.000010e+15,Citibank(R) Savings Plus Account,KYRA,DRAKE,319-77-3993,147 MILLER RD,PRESTON,CT,63658,Asian,Female
3,9.000010e+15,REGULAR CHECKING,WILLIAM,JONES,757-94-0576,2044 CROPSEY AVE,BROOKLYN,NY,11214,Native Hawaiian,Male
4,9.000010e+15,CitiGold Interest Checking,LORNA,CAMPOS,356-82-4395,2S653 PIERRE CURIE LN,WARRENVILLE,IL,60555,White,Male


In [6]:
print("SAMPLE TABLE 3")
print("--------------")
print("Rows:    {:4d}".format(sample_tables["table_3"].shape[0]))
print("Columns: {:4d}".format(sample_tables["table_3"].shape[1]))
sample_tables["table_3"].head()

SAMPLE TABLE 3
--------------
Rows:    1000
Columns:   14


,Institution Name,Branch Name,Established Date,Street Address,City,County,State,Zipcode,Latitude,Longitude,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits
0,JPMorgan Chase Bank,Renton Branch,10/20/1961,150 Logan Avenue South,Renton,King,WA,98057,47.48138,-122.20911,73082.0,73952.0,75749.0,72068.0
1,JPMorgan Chase Bank,Preston and Beltline Branch,11/06/2008,15114 Preston Rd,Dallas,Dallas,TX,75248,32.95420,-96.80397,7119.0,11684.0,22178.0,22365.0
2,JPMorgan Chase Bank,Hyde Park Banking Center Branch,12/15/2003,1204 East 53rd Street,Chicago,Cook,IL,60615,41.79954,-87.59649,49952.0,57415.0,65144.0,79133.0
3,JPMorgan Chase Bank,Nesconset Branch,04/17/1974,705 Smithtown Bypass,Smithtown,Suffolk,NY,11787,40.81819,-73.21205,23114.0,26757.0,32616.0,37519.0
4,JPMorgan Chase Bank,Novi Branch,05/28/1956,43100 Grand River Avenue,Novi,Oakland,MI,48375,42.48002,-83.47205,137078.0,145409.0,165753.0,181205.0


In [7]:
print("SAMPLE TABLE 4")
print("--------------")
print("Rows:    {:4d}".format(sample_tables["table_4"].shape[0]))
print("Columns: {:4d}".format(sample_tables["table_4"].shape[1]))
sample_tables["table_4"].head()

SAMPLE TABLE 4
--------------
Rows:    1000
Columns:   15


,institution,branch,date,street_name,city_name,county_name,state_name,Zip,Lat,Long,Deposits_2013,Deposits_2014,Deposits_2015,Deposits_2016,race
0,JPMorgan Chase Bank,Renton Branch,10/20/1961,150 Logan Avenue South,Renton,King,WA,98057,47.48138,-122.20911,72068.0,74047.0,81847.0,91001,American Indian
1,JPMorgan Chase Bank,Preston and Beltline Branch,11/06/2008,15114 Preston Rd,Dallas,Dallas,TX,75248,32.95420,-96.80397,22365.0,26223.0,62303.0,65061,Asian
2,JPMorgan Chase Bank,Hyde Park Banking Center Branch,12/15/2003,1204 East 53rd Street,Chicago,Cook,IL,60615,41.79954,-87.59649,79133.0,91040.0,108448.0,125273,African American
3,JPMorgan Chase Bank,Nesconset Branch,04/17/1974,705 Smithtown Bypass,Smithtown,Suffolk,NY,11787,40.81819,-73.21205,37519.0,48314.0,58648.0,65908,American Indian
4,JPMorgan Chase Bank,Novi Branch,05/28/1956,43100 Grand River Avenue,Novi,Oakland,MI,48375,42.48002,-83.47205,181205.0,202553.0,216597.0,259478,Asian


## Process
Here we demonstrate the primary function utilized by the __Emerald CLI__. As shown below, the model finds the data classification `data_class` for each column and a representation which is either a sparse vector of n-gram weights for `named_entity` or `text` data, or a regular expression for `numeric_or_id` data. The generated results are sent over for storage to the __Emerald Web Service__.

In [8]:
# Process all sample tables
results = []
for table_name, table_data in sample_tables.items():
    results.extend(emerald.process(table_data, max_sample_size=10, 
                                   identifiers={"table_name": table_name}))
emerald_internal_data = pd.DataFrame(results)
emerald_internal_data

,column_name,data_class,representation,table_name
0,CCID,numeric_or_id,^\d{16}$,table_1
1,ACCOUNT TYPE,text,"(0, 53)\t1.0\n (0, 86)\t1.0\n (0, 515)\t1....",table_1
2,SSN,numeric_or_id,^\d{9}$,table_1
3,GIVEN NAME,named_entity,"(0, 540)\t1.0\n (0, 550)\t1.0\n (0, 582)\t...",table_1
4,LAST NAME,named_entity,"(0, 193)\t1.0\n (0, 473)\t1.0\n (0, 481)\t...",table_1
5,PHYSICAL ADDRESS,named_entity,"(0, 256)\t1.0\n (0, 456)\t1.0\n (0, 540)\t...",table_1
6,CITY,named_entity,"(0, 266)\t1.0\n (0, 512)\t1.0\n (0, 548)\t...",table_1
7,STATE,named_entity,"(0, 1524)\t1.0\n (0, 1525)\t1.0\n (0, 1527...",table_1
8,POSTAL CODE,numeric_or_id,^\d{5}$,table_1
9,GENDER,text,"(0, 466)\t1.0\n (0, 7560)\t1.0\n (0, 8573)...",table_1


## Search
This similarity search function is run by the __Emerald Web Service__ to find all similar columns to a chosen query column. The results are filtered by a predefined score threshold, which is in turn separated into High, Medium or Low matches by the __Emerald Web Service__.

In [9]:
# Enter query here
query = "GENDER"

# Only run the query if it is a previously seen column name. 
# There is a validity check for this in the search bar of the frontend UI of the Emerald Web Service
if query in emerald_internal_data["column_name"].values:
    print(pd.DataFrame(emerald.search(query, emerald_internal_data, threshold=0.7)))
else:
    print("Invalid query. Please enter a valid column name.")

  column_name  score table_name
0      GENDER   1.00    table_1
1      P_GEND   0.82    table_2


In [10]:
# Enter query here
query = "CITY"

# Only run the query if it is a previously seen column name. 
# There is a validity check for this in the search bar of the frontend UI of the Emerald Web Service
if query in emerald_internal_data["column_name"].values:
    print(pd.DataFrame(emerald.search(query, emerald_internal_data, threshold=0.7)))
else:
    print("Invalid query. Please enter a valid column name.")

            column_name  score table_name
0                  CITY   1.00    table_1
1  P_PHYSICAL_CITY_NAME   0.96    table_2
2                  City   0.93    table_3
3             city_name   0.93    table_4


In [11]:
# Enter query here
query = "SSN"

# Only run the query if it is a previously seen column name. 
# There is a validity check for this in the search bar of the frontend UI of the Emerald Web Service
if query in emerald_internal_data["column_name"].values:
    print(pd.DataFrame(emerald.search(query, emerald_internal_data, threshold=0.7)))
else:
    print("Invalid query. Please enter a valid column name.")

       column_name  score table_name
0              SSN   1.00    table_1
1  SOCIAL_SECURITY   0.82    table_2
